In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the variables
host = os.getenv("OPENSEARCH_HOST", "localhost")
port = int(os.getenv("OPENSEARCH_PORT", 9201))
url = f"http://{host}:{port}"

auth = (
    os.getenv("OPENSEARCH_USER", "admin"),
    os.getenv("OPENSEARCH_PASSWORD", "admin"),
)
index_name = os.getenv("OPENSEARCH_INDEX_NAME", "langchain")

print(f"Host: {host}, Port: {port}, Auth: {auth}, Index: {index_name}")


Host: localhost, Port: 9201, Auth: ('admin', 'Supe5Senh@'), Index: langchain_graph_vector_1


In [2]:
import sys
sys.path.append("/Users/pedropacheco/Projects/dev/langchain/libs/community")


In [3]:
from langchain_community.graph_vectorstores.content_graph import ContentGraph


/Users/pedropacheco/Projects/dev/langchain/libs/community/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pathlib
g = ContentGraph("Lorem Ipsum")
g.fromPDFDocument(pathlib.Path("sample4.pdf"),output_image_path="./images", reset_graph=True, infer_hierarchy=True)
g.graph


In [ ]:
result = os_client.similarity_search_by_vector_and_metadata("sed diam nonummy nibh euismodtincidunt ut laoreet dolore magna aliquam erat volutpat", metadata = {"type": "Title"}, k=2)
for doc in results:
    print(doc.id, doc.page_content)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:opensearch:POST https://localhost:9201/langchain_graph_vector_1/_search?size=2 [status:200 request:0.025s]


3579e20ad32d793a8dd28ff5d8023da7 Your Company
35ccf72b73f67342fb97284be07e0de1 Product Brochure
e6cd9de5667b301da81d1373f90e734c Product Overview
002f87c6560b4f412ee064375b00319d Lorem ipsum
c20e14a2e6cc3abd9e3794c19db89c4e Lorem ipsum
73c2a9d151deb9c8f6b1113cb8faad74 Dolor sit
4836a5c73aaa4ee24183d49ebc83390f Details


In [5]:
from langchain_community.graph_vectorstores.opensearch import OpenSearchGraphVectorStore
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings()

os_client = OpenSearchGraphVectorStore(
    opensearch_url=url,
    http_auth=auth,
    index_name=index_name,
    embedding=embedding_model,
    reset_index=False,
)


/var/folders/hr/bsttdbns4nn9l1g54smlh_mr0000gn/T/ipykernel_28606/3963969764.py:5: LangChainBetaWarning: The class `OpenSearchGraphVectorStore` is in beta. It is actively being worked on, so the API may change.
  os_client = OpenSearchGraphVectorStore(


In [ ]:
os_client.aadd_content_graph(g)


In [7]:
os_client.similarity_search(
    "sed diam nonummy nibh euismodtincidunt ut laoreet dolore magna aliquam erat volutpat"
)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:opensearch:POST https://localhost:9201/langchain_graph_vector_1/_search?size=4 [status:200 request:0.105s]


[Document(id='935fdcf8ba2e601348691f23784106a0', metadata={'type': 'NarrativeText', 'links': {Link(kind='NarrativeText', direction='in', tag='e6cd9de5667b301da81d1373f90e734c')}, 'detection_class_prob': 0.9381677508354187, 'coordinates': {'points': [[197.91666666666666, 762.22021484375], [197.91666666666666, 897.2098526487088], [1442.4452049466554, 897.2098526487088], [1442.4452049466554, 762.22021484375]], 'system': 'PixelSpace', 'layout_width': 1700, 'layout_height': 2200}, 'last_modified': '2024-10-30T08:25:47', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'parent_id': 'e6cd9de5667b301da81d1373f90e734c', 'filename': 'sample4.pdf'}, page_content='Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat.'),
 Document(id='b58ded2c0d09fcd6b1810df1f7c

In [8]:
import asyncio

async def runner():
    async for document in os_client.asimilarity_search(
        "sed diam nonummy nibh euismodtincidunt ut laoreet dolore magna aliquam erat volutpat"
    ):
        print(document.id)


# Run the function directly if in an interactive environment like Jupyter Notebook
await runner()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:opensearch:POST https://localhost:9201/langchain_graph_vector_1/_search [status:200 request:0.029s]


935fdcf8ba2e601348691f23784106a0
b58ded2c0d09fcd6b1810df1f7cd9423
441d6021d20dac79ddd32d9e17568dc8
bc5e09f692d6cf4ac05d9e6d84ecf9ca


In [9]:
os_client.get_documents(k=10)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:opensearch:POST https://localhost:9201/langchain_graph_vector_1/_search [status:200 request:0.044s]


[Document(metadata={'type': 'Header', 'links': {Link(kind='Header', direction='in', tag='73c2a9d151deb9c8f6b1113cb8faad74')}, 'detection_class_prob': 0.7280164361000061, 'coordinates': {'points': [[197.3474578857422, 132.3511505126953], [197.3474578857422, 165.83247100000006], [216.63411466666665, 165.83247100000006], [216.63411466666665, 132.3511505126953]], 'system': 'PixelSpace', 'layout_width': 1700, 'layout_height': 2200}, 'last_modified': '2024-10-30T08:25:47', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 2, 'filename': 'sample4.pdf'}, page_content='2'),
 Document(metadata={'type': 'Title', 'links': {Link(kind='Image', direction='out', tag='b7cdb0075e34409ff9418ff94b025d74'), Link(kind='NarrativeText', direction='out', tag='441d6021d20dac79ddd32d9e17568dc8'), Link(kind='NarrativeText', direction='out', tag='ec4e566681062b3415e6e140502beb8b'), Link(kind='root', direction='in', tag='root')}, 'detection_class_prob': 0.8497406244277954, 'coordinates': {'points'

In [10]:
doc = os_client.search_by_id("bc5e09f692d6cf4ac05d9e6d84ecf9ca")
doc.page_content


INFO:opensearch:GET https://localhost:9201/langchain_graph_vector_1/_doc/bc5e09f692d6cf4ac05d9e6d84ecf9ca [status:200 request:0.025s]


'Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan.'

In [11]:
import asyncio


async def runner():
    doc = await os_client.asearch_by_id("bc5e09f692d6cf4ac05d9e6d84ecf9ca")
    print(doc.page_content)


# Run the function directly if in an interactive environment like Jupyter Notebook
await runner()


INFO:opensearch:GET https://localhost:9201/langchain_graph_vector_1/_doc/bc5e09f692d6cf4ac05d9e6d84ecf9ca [status:200 request:0.044s]


Duis autem vel eum iriure dolor in hendrerit in vulputate velit esse molestie consequat, vel illum dolore eu feugiat nulla facilisis at vero eros et accumsan.


{
  "took": 12,
  "timed_out": false,
  "_shards": {
    "total": 3,
    "successful": 3,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 100,
      "relation": "eq"
    },
    "max_score": 1.0,
    "hits": [
      {
        "_index": "your_index_name",
        "_type": "_doc",
        "_id": "1",
        "_score": 1.0,
        "_source": {
          "field1": "value1",
          "field2": "value2",
          "metadata": { ... }
        }
      },
      {
        "_index": "your_index_name",
        "_type": "_doc",
        "_id": "2",
        "_score": 0.9,
        "_source": {
          "field1": "value3",
          "field2": "value4",
          "metadata": { ... }
        }
      }
      // Additional hits...
    ]
  }
}

In [12]:
search_criteria = {"type": "Title"}
results = os_client.search_by_metadata(search_criteria, k=20)
for doc in results:
    print(doc.id)


INFO:opensearch:POST https://localhost:9201/langchain_graph_vector_1/_search?size=20 [status:200 request:0.052s]


3579e20ad32d793a8dd28ff5d8023da7
35ccf72b73f67342fb97284be07e0de1
e6cd9de5667b301da81d1373f90e734c
002f87c6560b4f412ee064375b00319d
c20e14a2e6cc3abd9e3794c19db89c4e
73c2a9d151deb9c8f6b1113cb8faad74
4836a5c73aaa4ee24183d49ebc83390f


In [13]:
import asyncio

async def runner():
    metadata = {"type": "Image"}
    async for document in os_client.asearch_by_metadata(metadata=metadata, k=5):
        print(document.id)

# Run the function directly if in an interactive environment like Jupyter Notebook
await runner()


INFO:opensearch:POST https://localhost:9201/langchain_graph_vector_1/_search?size=5 [status:200 request:0.051s]


8a137031615c0f20b771cfe3956d57f8
2c3e32aafc4e37c57c9299bb1abe8cf3
0c6140c07f5c2e92399007e8c762c4cb
b5906a2b13ee1bbfb62f054bd10f61f2
b7cdb0075e34409ff9418ff94b025d74


In [14]:
result = os_client.similarity_search_by_vector_and_metadata("sed diam nonummy nibh euismodtincidunt ut laoreet dolore magna aliquam erat volutpat", metadata = {"type": "Title"}, k=2)
for doc in results:
    print(doc.id, doc.page_content)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:opensearch:POST https://localhost:9201/langchain_graph_vector_1/_search?size=2 [status:200 request:0.025s]


3579e20ad32d793a8dd28ff5d8023da7 Your Company
35ccf72b73f67342fb97284be07e0de1 Product Brochure
e6cd9de5667b301da81d1373f90e734c Product Overview
002f87c6560b4f412ee064375b00319d Lorem ipsum
c20e14a2e6cc3abd9e3794c19db89c4e Lorem ipsum
73c2a9d151deb9c8f6b1113cb8faad74 Dolor sit
4836a5c73aaa4ee24183d49ebc83390f Details


In [15]:
result = os_client.traversal_search(
    "sed diam nonummy nibh euismodtincidunt ut laoreet dolore magna aliquam erat volutpat"
)

for doc in result:
    print(doc.id)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:opensearch:POST https://localhost:9201/langchain_graph_vector_1/_search?size=4 [status:200 request:0.016s]


page_content='Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh euismod tincidunt ut laoreet dolore magna aliquam erat volutpat. Ut wisi enim ad minim veniam, quis nostrud exerci tation ullamcorper suscipit lobortis nisl ut aliquip ex ea commodo consequat.' metadata={'type': 'NarrativeText', 'links': {Link(kind='NarrativeText', direction='in', tag='e6cd9de5667b301da81d1373f90e734c')}, 'detection_class_prob': 0.9381677508354187, 'coordinates': {'points': [[197.91666666666666, 762.22021484375], [197.91666666666666, 897.2098526487088], [1442.4452049466554, 897.2098526487088], [1442.4452049466554, 762.22021484375]], 'system': 'PixelSpace', 'layout_width': 1700, 'layout_height': 2200}, 'last_modified': '2024-10-30T08:25:47', 'filetype': 'application/pdf', 'languages': ['eng'], 'page_number': 1, 'parent_id': 'e6cd9de5667b301da81d1373f90e734c', 'filename': 'sample4.pdf'}
page_content='Lorem ipsum dolor sit amet, consectetuer adipiscing elit, sed diam nonummy nibh